In [29]:
#https://machinelearningmastery.com/time-series-prediction-lstm-recurrent-neural-networks-python-keras/
import pandas as pd
import json as json
from pandas import json_normalize
import os
import numpy as np
import csv
from matplotlib import pyplot as plt
from datetime import datetime

import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

numpy.random.seed(7)

# Load data

In [41]:
# load the dataset
df = pd.read_csv('../data/preprocessed/cleaned_data_v3.csv', delimiter=",")

# Prepare data

In [42]:
# set date as index ancd concert to float
df['datum'] = pd.to_datetime(df['datum'])
df = df.set_index(df['datum'])
df = df.sort_index()
df['datum_float'] = df['datum'].values.astype(float)

In [43]:
#drop cols
df = df.drop(columns=["height_delta", "height_increase", "datum"])

In [44]:
dataset = df.values
dataset = df.astype('float32')

In [52]:
df

,id,container_id,messungszähler,height_in_cm,temperatur_in_grad,tilt_in_grad,niederschlag_in_mm,laengengrad,breitengrad,ist_industriegebiet,...,DEVEUI_70B3D50070001781,DEVEUI_70B3D50070001782,DEVEUI_70B3D50070001786,DEVEUI_70B3D50070001787,DEVEUI_70B3D50070001788,DEVEUI_70B3D50070001789,farbe_brown,farbe_green,farbe_white,datum_float
datum,,,,,,,,,,,,,,,,,,,,,
2020-05-09 00:17:44.786185,597682,38,63,76,11.0,0,0.0,50.135236,8.708971,0,...,0,0,0,0,0,0,1,0,0,1.588983e+18
2020-05-09 00:18:34.664830,597684,58,131,90,19.0,1,0.0,50.129507,8.706566,0,...,0,0,0,0,0,0,0,0,1,1.588984e+18
2020-05-09 00:18:47.684715,597686,35,117,120,12.0,2,0.0,50.123940,8.762620,1,...,0,0,0,0,0,0,0,1,0,1.588984e+18
2020-05-09 00:19:06.064475,597688,55,235,100,20.0,0,0.0,50.133208,8.697431,0,...,0,0,0,0,0,0,0,0,1,1.588984e+18
2020-05-09 00:20:01.173813,597694,60,87,126,18.0,3,0.0,50.134389,8.711610,0,...,0,0,0,0,0,0,0,1,0,1.588984e+18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-08 22:21:17.974001,4111143,76,105,64,13.0,4,0.0,50.142071,8.698018,0,...,0,0,0,0,1,0,0,1,0,1.620512e+18
2021-05-08 22:21:31.255792,4111145,71,192,76,12.0,2,0.0,50.125186,8.710195,0,...,0,0,0,0,0,0,1,0,0,1.620512e+18
2021-05-08 22:21:52.093759,4111148,53,214,74,11.0,0,0.0,50.128183,8.704522,0,...,0,0,0,0,0,0,1,0,0,1.620513e+18


In [46]:
# normalize the dataset
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = scaler.fit_transform(dataset)

In [47]:
# create train test partition
train = df['2020-05-09 00:00:00':'2021-01-29 23:59:59']
test  = df['2021-01-30 00:00:00':]
print('Train Dataset:',train.shape)
print('Test Dataset:',test.shape)

Train Dataset: (383597, 100)
Test Dataset: (131440, 100)


In [48]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return numpy.array(dataX), numpy.array(dataY)

In [49]:
# reshape into X=t and Y=t+1
look_back = 1
trainX, trainY = create_dataset(train, look_back)
testX, testY = create_dataset(test, look_back)

TypeError: '(slice(0, 1, None), 0)' is an invalid key

In [ ]:
# reshape input to be [samples, time steps, features]
trainX = numpy.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
testX = numpy.reshape(testX, (testX.shape[0], 1, testX.shape[1]))